conda install numpy  
conda install pandas  
conda install matplotlib  
pip install SpeechRecognition  
pip install -U scikit-learn  
conda install -c anaconda nltk

In [1]:
import pydub
import pandas as pd
from tqdm import tqdm

pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

# Data loading

In [ ]:
def get_end_from_start(df_mail):
    end             = df_mail.iloc[1:,:]['start'].copy()
    end[len(end)+1] = -1
    df_mail['end']  = end.values
    return df_mail

def get_start_end_from_file(file):
    #Get start and end times
    df = pd.read_excel(file)
    df = df.rename(columns={'time':'start'}) 
    df['start'] = df['start']*1000 #in ms

    df = df.groupby('mail').apply(get_end_from_start)
    return df

def load_audios(video_folder,startend_file) :
    '''Loads the audios from each interview question.
    Arguments:
        video_folder : str. The name of the folder containing mp4 videos.
        startend_file : str. The name of the file containing the video informations. 
            Must contain columns 'mail' and 'time'.
    Returns a list of audios
    '''
    filenames = tqdm(os.listdir(video_folder))
    df_startend = get_start_end_from_file(startend_file)
    audios = list(map(lambda f : split_questions(video_folder,df_startend,f), filenames))
    return [item for sublist in audios for item in sublist]

In [3]:
from audio_utils import split_questions

video_folder = 'videos/'
df_name = 'test.xlsx'

audios = load_audios(video_folder,df_name)

100%|██████████| 4/4 [00:02<00:00,  1.37it/s]


# Lexical feature extraction

In [15]:
from speech_utils import speech_recognition

audios[0].export('temp.wav',format="wav")
result = speech_recognition('temp.wav')

In [16]:
from speech_feats_extract import FrenchStemTokenizer
from sklearn.feature_extraction.text import CountVectorizer

#WORD2VEC

In [17]:
countvect = CountVectorizer(tokenizer=FrenchStemTokenizer(remove_non_words=True))
text_fts = countvect.fit_transform([result])
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

countvect.get_feature_names()

C:\Users\Kinza\anaconda3\envs\pie\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['bien',
 'chos',
 'combat',
 'doit',
 'gestion',
 'moment',
 'pass',
 'pet',
 'plus',
 'pos',
 'pratiqu',
 'profit',
 'si',
 'sport',
 'station',
 'vi']